# HDBSCAN should work well on embedding representations

In [1]:
import os
import re
import json
import hdbscan

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from collections import Counter
import matplotlib.pyplot as plt

import lib.helper as helper
import lib.embedding_models as reps

from importlib import reload

%matplotlib inline

## 1.  Retrieve Corpus

The corpus is being scraped by the "run_news_scrapes.py" script (and windows task scheduler) every 12 hours, a bit past midday and a bit past midnight.

The "bing" corpus are news titles and text extracts gotten from the bing news search API, using a few Home Office - related keywords.

The "RSS" corpus is plugged directly into a number of RSS feeds for world news sites and local british news sites, with no filters for news story types or subjects applied.

In [2]:
# Should be same path for all my PC's, it's where each scrape goes as a separate json file.
storage_path = "D:/Dropbox/news_crow/scrape_results"

# "bing" is targeted news search corpus, "RSS" is from specific world and local news feeds.
corpus_type = "disaster"

# Load up
corpus = helper.load_clean_corpus(storage_path, corpus_type)

# Make sure after cleaning etc it's indexed from 0
corpus.reset_index(inplace=True)
corpus.index.name = "node"

# See how it turned out
print(corpus.shape)
corpus.head()

Total files: 315
9.8 percent of files read.
19.7 percent of files read.
29.5 percent of files read.
39.4 percent of files read.
49.2 percent of files read.
59.0 percent of files read.
68.9 percent of files read.
78.7 percent of files read.
88.6 percent of files read.
98.4 percent of files read.
(22786, 9)


,index,title,summary,date,link,source_url,retrieval_timestamp,origin,clean_text
node,,,,,,,,,
0,0,West Midlands <b>flood</b> warnings prompt &#3...,Residents have been warned to &quot;remain vig...,2019-11-17T17:35:00.0000000Z,https://www.bbc.co.uk/news/uk-england-50451817,www.bbc.co.uk,2019-11-17 19:50:58.278878,bing_news_api,West Midlands flood warnings prompt ;remain vi...
1,1,New <b>flood</b> warnings issued with more hom...,The Environment Agency has a number of <b>floo...,2019-11-17T18:35:00.0000000Z,https://www.hulldailymail.co.uk/news/hull-east...,www.hulldailymail.co.uk,2019-11-17 19:50:58.278928,bing_news_api,New flood warnings issued with more homes at r...
2,2,UK weather forecast – More than 100 <b>flood</...,<b>FLOOD</b>-ravaged villages in the UK have b...,2019-11-17T13:45:00.0000000Z,https://www.thesun.co.uk/news/10342583/uk-weat...,www.thesun.co.uk,2019-11-17 19:50:58.278953,bing_news_api,UK weather forecast – More than 100 flood aler...
3,5,UK <b>flood</b> warning map: <b>Flood</b> chao...,The Environment Agency has issued 57 <b>flood<...,2019-11-17T16:38:00.0000000Z,https://www.express.co.uk/news/weather/1205629...,www.express.co.uk,2019-11-17 19:50:58.279028,bing_news_api,UK flood warning map: Flood chaos to continue ...
4,6,UK weather forecast: <b>Flood</b> chaos contin...,Despite some areas enduring their &#39;wettest...,2019-11-17T18:32:00.0000000Z,https://www.mirror.co.uk/news/uk-news/uk-weath...,www.mirror.co.uk,2019-11-17 19:50:58.279047,bing_news_api,UK weather forecast: Flood chaos continues wit...


## 2.  Build Text Model (Representation, eg; word2vec, entities list...)

- Trying with the world corpus and with the bing corpus, neither worked with InferSent.  Suspect the problem lies in the PCA step, which may not be working well on this high-dimensional (vector length = 4096) form.
- Summed keywords works rather better with the world corpus.
- Summed keywords still fail the bing/home office corpus, giving me a cluster about "immigration" and a cluster for the American Supreme Court.

In [3]:
# Windows didn't play nicely with the vector datasets, Some obscure encoding problem (python in Conda
# kept trying to decode using cp1252 regardless of whatever other options I specified!)
# Solution; rewrite file and drop any characters the Windows encoder refuses to recognise.
# I shouldn't loose too much info.
#with open('./lib/InferSent/dataset/fastText/crawl-300d-2M.vec', "r", encoding="cp1252", errors="ignore") as infile:
#    with open('./lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec', "wb") as outfile:
#        for line in infile:
#            outfile.write(line.encode('cp1252'))

In [4]:
#infersent = reps.InferSentModel(list(corpus['clean_text']),
#                                list(corpus['clean_text']),
#                                W2V_PATH = './lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec')

#embeddings = infersent.get_embeddings()

In [5]:
#reload(reps)

In [6]:
# Whereas this worked first time!
#glove = reps.NounGloveWordModel(list(corpus['clean_text']), list(corpus['clean_text']))

#embeddings = glove.get_embeddings()

In [7]:
# Turn that into a DF for me
#embeddings_df = pd.DataFrame({"clean_text": list(embeddings.keys()),
#                              "embeddings": list(embeddings.values())})
#embeddings_df.shape

In [8]:
#embeddings_df.head()

## 2a.  Try a really simple averaged word vector model!

With a complex noun extraction function 'cause that part's slow so I multi-threaded it.

In [65]:
from gensim.models import Word2Vec
import spacy
nlp = spacy.load('en_core_web_sm')
from gensim.models.phrases import Phrases, Phraser

# Set vector size
vec_size = 50

In [66]:
def get_phrased_nouns(sentences):
    """ Use spacy to get all of the actual entities, conjoin bigram nouns. """

    # Get the lists of nouns
    noun_lists = []
    for doc in sentences:
        parsed = nlp(doc)
        noun_lists.append([token.lemma_ for token in parsed if token.pos_ == 'PROPN'])

    # Build the phrase model
    phrases = Phrases(noun_lists, min_count=5, threshold=0.5)

    # Get the set of phrases present in the model
    results = []
    for nouns in noun_lists:
        results.append(phrases[nouns])

    return results

# Get phrase-conjoined, lemmatized tokens
test = get_phrased_nouns(corpus['clean_text'])

# Detect and conjoin bigrams
model = Word2Vec(test, size=vec_size, window=5, min_count=1, workers=10)

In [67]:
def get_averaged_vec(token_list, model):
    
    vecs = []
    for token in token_list:
        try:
            vector = model.wv[token]
        except: 
            vector = np.zeros(vec_size)
        vecs.append(vector)
    
    if len(vecs) > 0:
        return np.mean(np.asarray(vecs), axis=0)
    else:
        return np.zeros(vec_size)

In [68]:
vectors = [get_averaged_vec(tokens, model) for tokens in test]

In [69]:
vectors[0:2]

[array([ 0.02786469, -0.04616074, -0.07180868,  0.21488126,  0.3411784 ,
        -0.12574303,  0.0269457 ,  0.05665186, -0.08210266,  0.00741011,
         0.04765641, -0.4026303 , -0.01547079,  0.09816053, -0.04588069,
         0.11431786,  0.16667897,  0.1571284 , -0.01724462, -0.25126076,
        -0.08565275,  0.05438086,  0.20457466,  0.2243854 , -0.20546375,
        -0.10605477, -0.05474118,  0.19553265, -0.05931751, -0.10219716,
         0.10303677,  0.00345081,  0.04938901, -0.26121497,  0.08799208,
        -0.01091591,  0.07456185,  0.22420777, -0.21982795,  0.19364956,
        -0.00793161,  0.0007707 ,  0.24674517, -0.11754436,  0.03463756,
        -0.19711307, -0.03607025, -0.20056218,  0.10731871,  0.07274119],
       dtype=float32),
 array([ 1.1440734e-01, -2.3599108e-01, -3.8487992e-01,  1.1108600e+00,
         1.7454450e+00, -6.1501479e-01,  1.6115594e-01,  2.6300907e-01,
        -4.2414147e-01,  6.0580857e-03,  2.3323029e-01, -2.0551350e+00,
        -6.7068920e-02,  5.071

## 3. Cluster Text

This is the part where the pipelines get a little more experimental
- First; PCA (HDBSCAN prefers < 50 dimensions if possible) (also, try just fitting 50 vectors!)
- Second; HDBSCAN clustering

In [70]:
embeddings_array = np.vstack(vectors)

# First, PCA the data
#pca = PCA(n_components=20, svd_solver='full')

# Fit and check
#embeddings_pca = pca.fit_transform(embeddings_array)

# Diagnostic stats - both should show exponential decay
#print(pca.explained_variance_ratio_)
#print(pca.singular_values_) 

In [71]:
# Clusterer fitting
clusterer = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=10)
clusterer.fit(embeddings_array)

# Examine results
Counter(clusterer.labels_)

Counter({54: 11813,
         -1: 9349,
         1: 38,
         5: 9,
         51: 9,
         31: 12,
         53: 313,
         9: 13,
         24: 18,
         10: 19,
         26: 5,
         46: 7,
         35: 5,
         8: 5,
         17: 11,
         47: 9,
         25: 22,
         21: 10,
         18: 8,
         22: 13,
         30: 5,
         12: 7,
         34: 23,
         23: 10,
         40: 8,
         4: 12,
         52: 5,
         0: 103,
         50: 237,
         28: 103,
         49: 54,
         27: 17,
         13: 19,
         20: 16,
         43: 39,
         15: 13,
         36: 27,
         48: 28,
         6: 46,
         7: 76,
         11: 15,
         37: 17,
         29: 22,
         19: 5,
         14: 16,
         39: 17,
         32: 24,
         42: 19,
         45: 29,
         41: 8,
         16: 5,
         2: 35,
         38: 14,
         44: 6,
         3: 6,
         33: 12})

In [72]:
# Diagnostic - calculate percentage of records assigned "outlier"
100.0 * sum(clusterer.labels_ == -1) / clusterer.labels_.shape[0]

41.02957956640042

In [73]:
len(pd.unique(clusterer.labels_))

56

For disaster, 50-len W2V direct, 156 clusters, 75 % outlier
For disaster, 50-len W2V pca-d, 220 clusters, 77 % outlier
For disaster, 100-len W2V direct, 220 clusters, 73 % outlier
For disaster, 100-len W2V pca-d, 248 clusters, 73 % outlier

In [17]:
# Record cluster each was assigned to
corpus['cluster'] = clusterer.labels_

# Record the reverse of outlier score (therefore, higher number = higher certainty of membership)
corpus['score'] = 1.0 - clusterer.outlier_scores_

corpus.to_csv("working/disaster_clustered_w2v_direct.csv", index=False)

C:\ProgramData\Anaconda3\lib\site-packages\hdbscan\hdbscan_.py:1052: RuntimeWarning: invalid value encountered in double_scalars
  self._outlier_scores = outlier_scores(self._condensed_tree)
